**Introduction** 

This Appendix only includes the prompts that were used in our `Do Large Language Models understand literature? Case studies and probing experiments on German poetry.` 
For the models' output see the the jupyter notebooks referenced as the data and code repository or: https://github.com/cophi-wue/llms_read_hoelderlin

After some pre-studies we decided not to use open source models like Llama 3.1:70b, because they didn't perform as well as the commercial models offered by OpenAI, Google and Anthropic. Llama 3.1:405b wasn't an option because we didn't have access to an infrastructure able to run it. 

Our selection was based on evaluations of these models on Chatbot Arena (1.11.2024): https://lmarena.ai/ 

List of LLMs used: 

* OpenAi: ChatGPT-4o (gpt-4o-2024-08-06)
* Anthropic Claude 3.5 Sonnet 2024-10-22
* Google Gemini 1.5 Pro (Sep 2024)


**Two texts**

We are using two texts. One is a very well-known poem, 'Hälfte des Lebens' ('The middle of life') by Friedrich Hölderlin, a very famous German poet from the early 19th Century. There are many representations of this poems in German and English online and also many interpretations. 
The other poem, 'Unsere Toten', by Hans Pfeifer, is unknown to Google at the time of writing and has not been published again after its first publication 1922. The author is also not a known figure in literary history. 

* How well do the models know Hölderlin's text?
* Does it matter, whether a model knows a text or not? Is the information used when producing new text about the reference text?
* How well does the recognition of the text type work?
* How relevant is the information about the text type?

**The Poems to be analyzed in the paper:**

**Friedrich Höderlin: Hälfte des Lebens (1804)**

Mit gelben Birnen hänget 

Und voll mit wilden Rosen 

Das Land in den See, 

Ihr holden Schwäne, 

Und trunken von Küssen 

Tunkt ihr das Haupt 

Ins heilignüchterne Wasser.

--

Weh mir, wo nehm’ ich, wenn 

Es Winter ist, die Blumen, und wo 

Den Sonnenschein, 

Und Schatten der Erde? 

Die Mauern stehn 

sprachlos und kalt, im Winde 

klirren die Fahnen.

--

First published in 1804 in Friedrich Wilman's "Taschenbuch für das Jahr 1805"

In: Friedrich Hölderling: Gesammelte Werke. Herausgegeben von Hans Jürgen Balmes. Frankfurt am Main: Fischer, p. 163.

**Hans Pfeifer: Unsere Toten (1922)**

Von Westen und Osten, von Nord und Süd

Schleppen sich nächtens viele Füße müd,

Füße, vom Wandern wund und zerfetzt,

Langsam bedächtig zur Erde gesetzt,

Müh'n sich im zitternden Mondenschein

Rastlos tief nach Deutschland hinein.

Und wer mit lauschendem Ohr noch wacht

Hört sie in jedweder werdenden Nacht,

Hört dies Schlurfen so müde und schwer,

Hört eine Klage voll wilder Begehr,

Eine Klage schmerzzerfressen:

Nur nicht vergessen! Uns nicht vergessen!


In: Uhlmann-Bixterheide, Wilhelm, ed. (1922). Die deutsche Balladen-Chronik. Ein Balladenbuch von deutscher Geschichte und deutscher Art. Dortmund: Ruhfus, p. 290. 

The texts and several translations into English can be found in the `definitions.py`.

# Meter

## Configuration

In [ ]:
import os

from definitions import poem_1, poem_2
from utils import settings, gemini, gpt4, opus, init_gemini, printmd
import utils

%load_ext jupyter_ai_magics

In [ ]:
#settings
temperature = 0.8
system_prompt = "You are an expert in German literature. You answer the questions truthfully and short."

settings(system_prompt, temperature)
    

In [ ]:
#defining aliases
init_gemini()

model =  gpt4()
%ai register gpt4o model

model = opus()
%ai register opus model 

## Metrics

### Hälfte des Lebens

#### Level 1: General Knowledge

On this level we just ask about the metrical structure of the poem.

Solution(s): 
Each stanza has seven lines, of which three lines have three stresses and four have two.
First stanza: Of the lines with three stresses in the first stanza, a pair is placed at the beginning, and a single one at the end: The longer lines with three stresses frame the shorter ones. (Strauss assumes that the line "Es Winter ist, die Blumen, und wo" has three consecutive unstressed syllables, namely "ter ist, die".)

Second stanza also begins with two three-stressed lines. The third three-stressed line is the penultimate one.
In short:

1. Str: 3-3-2-2-2-2-3
2. Str: 3-3-2-2-2-3-2

In [ ]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two stanzes each with two lines 
"The house is green
The mouse is dead. 

I wonder still
Who pays my bill."

The output would look like this:
1: -/-/
2: -/-/
3: //-/
4: //-/

At the end repeat the numbers of stressed sylables for each line for each stanza like this:

Stanza 1: 2-2
Stanza 2: 3-3

Here is the poem: 
{poem_1.text}
"""
print(prompt)

##### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

##### Anthropic Claude 3.5 Sonnet

In [ ]:
%%ai opus
{prompt}

##### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

##### Evaluation and Discussion

The task  actually demands two steps: first detect the scansion, second report it in a summary way. Some of models have some difficulties to detect the correct scansion: Sonnet get's it right most of the time, while GPT-4o and Gemini 1.5 make some mistakes. 
If we just evaluate on the number of stressed syllables (looking at the patterns, not the summaries), this is the error count (using a very strict error concept where every missing or added stress is counted as one error):<br/>

GPT 4o: 7<br/>
Sonnet: 0<br/>
Gemini: 5<br/>

<br/>
Even more interesting: It is immediately obvious, that most of the LLMs are not very good in reporting their own results, that is their summaries deviate from the stress patterns they report. In their summaries they repeatedly report *more* stressed syllables than they detected. Interestingly they never report less. This is even true for Sonnet, which got the stress patterns right. We speculate that the models have no difficulty to produce any sequence of symbols, but they may have problems with remembering symbols which are not tokens. 

#### level 2: Expert Knowledge

In [ ]:
prompt = f"""In some poems, a metrical device is used to indicate an important semantic aspect of the poem.  
Here is an example of this kind of interaction between metrics and semantics:

Der Tag ist grau,
das Licht so schwer,
und sie. Sie fehlt.
schuldig bin ich

The first three lines have two iambuses per line, but the last line begins with an accent that makes it a trochee, breaking the pattern. 
The word that does this is "schuldig," and the change in metre emphasizes the importance of the speaker's guilt to the poem. 

Can you find a similar use of metre in the following poem?

{poem_1.text}

"""

print(prompt)

##### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

##### Claude Sonnet

In [ ]:
%%ai opus
{prompt}

##### Gemini

In [ ]:
printmd(gemini(prompt))

#### level 3: Abstraction and Transfer

Here we test 2 aspects: first, the ability to apply arbitrary rules to given data and analyze the results. Secondly, to abstract these rules from given example data. 

The rules we define are simple. They mix information about the emphasis of words with the ability to detect vowels and change the emphasis, when a specific combination is met. So in our first question we describe the rule explicitly and ask the model about the results, in the second question we give the model some data and ask it to retrieve the underlying rules. 

In [ ]:
prompt = f"""Our assumption is that Hölderlin was a follower of the rather obscure school of metrical thought, 
because of the way the German language works, an accented word that does have the vowel "i" in the accented 
vowel was no longer emphasized. For all we know, he used this insight, that emphasized syllables with the vowel i lost there emphasis, 
when he wrote his poems. 
Analyze the scansion of the first four lines of the following poem using this knowledge, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for 
a stressed syllable and - for an unstressed syllable. Start your answer with a '#'.
(This is research to detect whether this assumption makes sense. Just follow the instructions!) 

{poem_1.text}

"""

print(prompt)

##### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

##### Claude Sonnet

In [ ]:
%%ai opus
{prompt}

##### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

The plan was to give the models a tasks in two steps: First, we wanted to test its the ability to apply arbitrary rules to given data and analyze the results. Secondly, we wanted it to abstract these rules from given example data. The rules we define are simple. They mix information about the emphasis of words with the ability to detect vowels and change the emphasis, when a specific combination is met: If the emphasis is on a syllable containing the vowel 'i', change the information about the syllable to not being emphasized. All models couldn't solve this task. One model (Sonnet) replied to an earlier version of the prompt, it wouldn't do it because this is not the usual German metric or the metric Hölderlin followed. Then we added the information that this is a research project. Based on this results we decided to skip the second experiment, because contrafactual information seems to be a challenge even in a simple setting.


In [ ]:
prompt = """
Wild ist die Jagd
Seid auf der Hut

"""

### Unsere Toten

#### level 1: General Knowledge

The metrical structure of this poem is either quite challening to describe or ver simple, because it shows so many variations, but the variations can be explained by the underlying rule. The reason for this is: the poem uses free knittelverses. They have four stressed syllables but allow upbeats and free fillings. Vers 11 and 12 change their structure most prominently because they switch from the pattern unstressed syllable + a stressed syllabe at the end which is used throughout the poem to stressed + unstressed, a change which is emphasized by the fact that these are also rhyme words. 

```
1) -/--/--/-/
2) /--/---/-/
3) /--/-/--/
4) /--/--/--/
5) /--/--/-/
6) /-/-/--/
7) -/-/--/-/
8) /--/--/--/
9) /-/--/--/
10) /--/--/--/
11) /-/-/-/-
12) -/-/-/--/-
```

In [ ]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 
{poem_2.text}
"""
print(prompt)

##### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

##### Claude Sonnet

In [ ]:
%%ai opus
{prompt}

##### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

#### Level 2: Expert Knowledge

In [ ]:
prompt = f"""In some poems, a metrical device is used to indicate an important semantic aspect of the poem.  
Here is an example of this kind of interaction between metrics and semantics:

Der Tag ist grau,
das Licht so schwer,
und sie. Sie fehlt.
schuldig bin ich

The first three lines have two iambuses per line, but the last line begins with an accent that makes it a trochee, breaking the pattern. 
The word that does this is "schuldig," and the change in metre emphasizes the importance of the speaker's guilt to the poem. 

Can you find a similar use of metre in the following poem?

{poem_2.text}

"""

print(prompt)

##### GPT-4o

In [ ]:
%%ai gpt4o
{prompt}

##### Gemini

In [ ]:
printmd(gemini(prompt))

##### Claude Sonnet

In [ ]:
%%ai opus
{prompt}

##### another question
The  Knittelvers is a German vers structure which may be harder to detect, because it only counts the stressed syllables and ignores the unstressed syllables. In other words, the Knittelvers is defined by its free filling of unstressed syllables (variable number of syllables). 

In [ ]:
prompt = f"""This poem follows a  German verse meter. Which one?

{poem_2.text}"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
# a veriation of the last question.
prompt = f"""The following poem follows a verse meter. Which one?

{poem_2.text}"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

#### Level 3: Abstraction and Transfer

### Language bias in metrical analysis

In [ ]:
t = """
One face looks out from all his canvases,
One selfsame figure sits or walks or leans:
We found her hidden just behind those screens,
That mirror gave back all her loveliness.
A queen in opal or in ruby dress,
A nameless girl in freshest summer-greens,
A saint, an angel—every canvas means
The same one meaning, neither more or less.
He feeds upon her face by day and night,
And she with true kind eyes looks back on him,
Fair as the moon and joyful as the light:
Not wan with waiting, not with sorrow dim;
Not as she is, but was when hope shone bright;
Not as she is, but as she fills his dream.
"""

In [ ]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two stanzes each with two lines 
"The house is green
The mouse is dead. 

I wonder still
Who pays my bill."

The output would look like this:
1: -/-/
2: -/-/
3: //-/
4: //-/

At the end repeat the numbers of stressed sylables for each line for each stanza like this:

Stanza 1: 2-2
Stanza 2: 3-3

Here is the poem: 
{t}
"""


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
print(gemini({prompt}))

#### Results
* llama3 doesn't work at all (the lines are chopped and therefore the number of stressed sylables is false)
* GPT4o falsely sees six stressed sylables in the detail analysis in vers 1, 2, 11. And it doesn't get the two trochees at the end.  
* Opus doesn't recognize the pentameter and doesn't see the trochees in the last two lines.
* Gemini doesn't see the pentameter and changes its count between 4 and 5 stresses, but it gets the trochees. 
* Btw: Gpt4o splits the poem into an octet and a sextet, which is actually true for the Petrarchan sonnet, but not for this poem, which has only one stanza. 


### Can large language models count?

In [ ]:
prompt = f"""In the following poem count the number of vowels for each word in a vers. Report the results in the following fashion: 

1: 4 2 1 3
2: 2 4 1 2 
 etc. 

At the end report the number of vowels per line for each stanza like this:

1. stanza: 10 - 9 
2. stanza: 9 - 11

If the first two stanzas look like this: 

"It happens to the best of us 
that life feels like a passing bus

and we stay here and there 
it goes and leaves."

the answer would be:

1: 1 2 1 1 1 1 1 
2: 1 2 3 2 1 2 1 

3: 1 1 1 2 1 2 
4: 1 2 1 3

1. stanza: 8 - 12
2. stanza: 8 - 7

Here is the poem: 

{poem_1.text}
"""
print(f"{prompt}")

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
print(gemini({prompt}))

No, they cannot.

## Results and Discussion

The metrical analysis of both poems shows across the models a surprisingly high amount of errors. The answers also show, that the models have difficulties to remember their own results. These errors are not caused by the fact that we analyze German poems, but can also be seen in a quite famous English poem. 

## Configuration

In [ ]:
from definitions import poem_1, poem_2
from utils import settings, gemini, gpt4, opus, init_gemini, printmd
import utils

%load_ext jupyter_ai_magics 

In [ ]:
#settings
temperature = 0.8
system_prompt = "You are an expert in German literature and you are addressing other experts in German literature. You answer the questions truthfully and short."

settings(system_prompt, temperature)


In [ ]:
#defining aliases
init_gemini()

model =  gpt4()
%ai register gpt4o model

model = opus()
%ai register opus model 

# Rhyme



## Hälfte des Lebens

### Level 1: General Knowledge

We are just asking for the detection of rhymes in the poem. 
First we want to know whether there are any (end) rhymes. If yes, which words are rhymed and finally what kind of rhyme schema there is, like ABAB or AABB or ABBA etc.

Correct answer: 
There are no rhymes in 'Hälfte des Lebens' only many assonances and consonances. 

In [ ]:
prompt = f"""Analyze the (end) rhymes in this poem: \n {poem_1.text}. 

Does the poem use rhymes for all or most of its verses? 
If yes, what words are rhymed and what type of rhyme scheme does the poem use (like AABB or ABAB or ABBA etc.)? 
"""

print(prompt)


#### GPT-4o

In [ ]:
%%ai gpt4
{prompt}

#### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

#### Claude Sonnet

In [ ]:
%%ai opus
{prompt}

### Level 2: Expert Knowledge

As there is no rhyme in HdL, there is no meaningful expert question to be asked (False answer would probably mostly be false because the falsely detect rhymes).



### Level 3: Abstraction and Transfer

As rhyme is just a special case of assonances, see the complex probing experiments there.

## Unsere Toten

### Level 1: General Knowledge

We are just asking for the detection of rhymes in the poem. 

Correct answer: 


In [ ]:
prompt = f"""Analyze the (end) rhymes in this poem: \n {poem_2.text}. 

Does the poem use rhymes for all or most of its verses? 
If yes, what words are rhymed and what type of rhyme scheme does the poem use (like AABB or ABAB or ABBA etc.)? 
"""

print(prompt)


Correct answer: 
The poem has a parallel rhyme schema (AABB). So the words 'Süd' and 'müd' are rhymes, 'zerfetzt' and 'gesetzt' etc.  

#### Gpt-4o

In [ ]:
%%ai gpt4
{prompt}


#### Gemini 1.5

In [ ]:
printmd(gemini(prompt))


#### Sonnet 

In [ ]:
%%ai opus
{prompt}

### Level 2: Expert Knowledge

Can we detect an additional semantic structure, an additional layer of meaning, if we just look at the rhyme words of the poem? What would be plausible interpretative strategies and do they work (i.e. are they productive, do they produce any additional insights) for this poem?

Expected answer: Two possible directions of analysis could be expected: 
1) one follows the meaning which is etablished by pairing words in a rhyme. There is not a lot of this happening in the first half, but in the second half, we see that 'wacht' and 'Nacht' belong somewhat to opposites, similarly 'schwer' (which recalls the 'müd' (tired)) and 'Begehr'. Finally the last rhyme 'zerfressen' (eaten away) and vergessen (forgotten) implies an even stronger semantic relation, but not of opposites, rather both imply that there has been something which is no getting lost in a painful process. 
2) The second looks at the vowels of the rhymes and connects them to the tone and mood of the poem. We cannot detect anything which would be the basis for an interesting and productive description. 

In [ ]:
prompt = f"""Describe at least two strategies to analyze the rhyme structure of poems in relation to the meaning of the poem or parts of the poem. 
Then analyze whether they produce interesting insights when applied to the following poem:

{poem_2.text}. 

"""

print(prompt)


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
%%ai opus
{prompt}

### Level 3: Abstraction and Transfer

As rhyme is just a special case of assonances, see the complex probing experiments there.

# Assonance

## Hälfte des Lebens

### Task 1: Student communication: simple identification task: Detect assonances
#### 1.a: Detection based on different definitions of assonance
#### 1.b: Detection based on a two-step CoT-prompt: 1) Translate to IPA, 2) then perform assonance detection
#### 1.c: Same tasks for English¶

#### Expectation:
- Definition dependency: The more precisely the sound and phoneme characteristics are explained, the better the recognition. Assonances
- Difficulties in recognizing the correct sounds in German. 

In [ ]:
assonance = """Assonance is a similarity in sound between two stressed syllables 
that are close together in a text, created by the same vowels but different consonants """

assonance02 = """Assonance is the repetition of identical or similar phonemes in words or syllables that occur close together
in terms of their vowel phonemes (e.g. lean green meat)"""

assonance03 = """Assonance is the repetition of identical or similar phonemes between two stressed syllables that occur close together
in terms of their vowel phonemes (e.g. lean green meat). Note that identical vowels do not always imply same phonems. 
The 'o'-sound in 'often' and 'other' does not produce the same phonem"""

In [ ]:
prompt01 = f"""Here is a definition of assonance: {assonance}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.text} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use the following form in your answer: 'Dem Nordmann schwinden die Sorgen' - the 'o' sound in 'Nordmann' and 'Sorgen' creates assonance."""

prompt02 = f"""Here is a definition of assonance: {assonance02}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.text} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use the following form in your answer: 'Dem Nordmann schwinden die Sorgen' - the 'o' sound in 'Nordmann' and 'Sorgen' creates assonance."""

prompt03 = f"""Here is a definition of assonance: {assonance03}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.text} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use the following form in your answer: 'Dem Nordmann schwinden die Sorgen' - the 'o' sound in 'Nordmann' and 'Sorgen' creates assonance."""

prompt04_ipa = f"""Here is a definition of assonance: {assonance03}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.text} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use a two-step reasoning: Firstly, translate the poem to IPA. Secondly, perform the requested analysis of assonances based on the IPA-translation.
Use the following form in the second step of your answer: 'Dem Nordmann schwinden die Sorgen' - the 'o' sound in 'Nordmann' and 'Sorgen' creates assonance."""

prompt_en01 = f"""Here is a definition of assonance: {assonance}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.translations[0]} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use the following form in your answer: 'The light of the fire is a sight.' - the long 'i' sound in 'light', 'fire, and 'sight' creates assonance."""

prompt_en02 = f"""Here is a definition of assonance: {assonance02}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.translations[0]} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use the following form in your answer: 'The light of the fire is a sight.' - the long 'i' sound in 'light', 'fire, and 'sight' creates assonance."""


prompt_en03 = f"""Here is a definition of assonance: {assonance03}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.translations[0]} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use the following form in your answer: 'The light of the fire is a sight.' - the long 'i' sound in 'light', 'fire, and 'sight' creates assonance."""

prompt04en_ipa = f"""Here is a definition of assonance: {assonance03}. 
Using this definition, give a full description of all assonances in this poem: \n {poem_1.translations[0]} \n
Explain in each case the assonance by repeating the words and by giving the common vowel.
Use a two-step reasoning: Firstly, translate the poem to IPA. Secondly, perform the requested analysis of assonances based on the IPA-translation.
Use the following form in your answer: 'The light of the fire is a sight.' - the long 'i' sound (IPA: aɪ) in 'light', 'fire, and 'sight' creates assonance."""

## Task 1.a

In [ ]:
print(prompt01)

## Gold-Standard: alle Assonanzen:
### Stanza 1
Z 1: Mit / Birnen

Z 1: gelben / hänget

Z 2: mit / wilden

Z 3: den / See

Z 4: Und / trunken ( + Z 5  Tunkt)

Z 5 (s. 4 tunkt) 

Z 6

Z 7

### Stanza 2

Z. 8 Weh / nehm

Z. 8 Winter / ist

Z 9. - 

Z 10 - 

Z 11 der / Erde

Z 12 -

Z 13 - Winde (bis Z. 14 Klirren)

Z. 14 

In [ ]:
%%ai gpt4o
{prompt01}

In [ ]:
print(prompt02)

In [ ]:
%%ai gpt4o
{prompt02}

In [ ]:
print(prompt03)

In [ ]:
%%ai gpt4o
{prompt03}

In [ ]:
%%ai gpt4o
{prompt04_ipa}

In [ ]:
print(prompt_en01)

Assonances gold standard englisch

l 1: yellow / pears
l 2: -
l 3: land / hangs / lake
l 4 o / lovely 
l5 with kisses
l 6 you your
l 7 -

l 8 alas / shall (naja)
l 9 when / where
l 10 - 
l11 and / shadows 
l 12 
l 13 in / wind
l 14 waether/vanes/clatter


In [ ]:
%%ai gpt4o
{prompt_en01}

In [ ]:
print(prompt_en02)

In [ ]:
%%ai gpt4o
{prompt_en02}

In [ ]:
print(prompt_en03)

In [ ]:
%%ai gpt4o
{prompt_en03}

In [ ]:
%%ai gpt4o
{prompt04en_ipa}

In [ ]:
print(prompt01)

printmd(gemini(prompt01))

In [ ]:
printmd(gemini({prompt02}))

In [ ]:
printmd(gemini(prompt03))

In [ ]:
printmd(gemini(prompt04_ipa))

In [ ]:
printmd(gemini(prompt_en01))

In [ ]:
printmd(gemini(prompt_en02))

In [ ]:
printmd(gemini(prompt_en03))

In [ ]:
print(prompt04en_ipa)

In [ ]:
printmd(gemini(prompt04en_ipa))

In [ ]:
print(prompt01)

In [ ]:
%%ai opus
{prompt01}

In [ ]:
%%ai opus
{prompt02}

In [ ]:
%%ai opus
{prompt03}

In [ ]:
print(prompt_en01)

In [ ]:
%%ai opus
{prompt_en01}

In [ ]:
%%ai opus
{prompt_en02}

In [ ]:
%%ai opus
{prompt_en03}

## Table: Summary of the qualitative results für Hölderlin's "Hälfte des Lebens

| Model  | Recall (de) | Precision (de) | max. precision english | prompt for max recall |
|--------|-------------|----------------|------------------------|-----------------------|
| GPT4o  | 0.2         | 0.2            | 0.83                   | Def 1                 |
| Gemini | 0.2         | 0.2            | 0.44                   | Def 2                 |
| Sonnet | 0.2         | 0.35           | 0.75                   | Def 1                 |

Recall and Precision for Assonances in the German version of Hölderlin's "Hälfte des Lebens" with n~10 instances of true assonance occurrences.

## Task 2: Epert communication: interpretation based on assonances


In [ ]:
prompt_expert = f"""Perform the following task in two steps. 
Step 1: {prompt04_ipa}
Step 2: Based on step 1, give an interpretation of the poem that takes into account the connections and contrast between the semantic fields that 
are grouped by the different assonances.
"""


In [ ]:
%%ai gpt4o
{prompt_expert}

In [ ]:
printmd(gemini(prompt_expert))

In [ ]:
%%ai opus
{prompt_expert}

## Summary of the expert task

Since the results on the "General knowledge" level showed considerable weaknesses, we did not refer the results of the "Expert knwoledge" level in the paper. Nevertheless, it is instructive to force the models to perform more complex tasks. The task we designed was to provide an interpretation of the poem that tries to relate the words that are connected by assonance on the level of textual meaning by grouping and contrasting assonance-based semantic fields. Again, all models failed to correctly detect assonance. However, in cases with correct connection, surprisingly good hypotheses were raised – for instance by Gemini, which based its interpretation on the assonance claiming for the second stanza of Hölderlin's poem that "the 'eː' sound links the speaker's lament ("Weh mir") with the absence of summer elements ("nehm’," "stehn"). Though not an entirely precise analysis, the connection between lament and privation is reasonable.
%(it would, of course, have been more precise to interpret the verb "nehmen" here as referring to the problem of compensating the privation of summer). 
Here we see that the models perform well at the seemingly more complex tasks of seeing non-trivial and latent semantic relations and even of combining different latent relations. Since the basic description of the sound characteristics of the poem lacks precision, further interpretive hypotheses on the level of expert knowledge have not yet become convincing, however.

### Task 3: Abstraction-Transfer
#### 3.a Counterfactual Assonances: The notion of "eusonance": Detecting eusonances

In [ ]:
eusonance = """Eusonance occurs when the vowel sounds of the stressed syllables in two consecutive words in the same line are a German i-sound and a German e-sound. 
The order between the i sound and the e sound does not matter.
Monosyllabic words always count as words with a stressed syllable
(e.g. "Ich" and "esse" in "Ich esse Kuchen")."""

In [ ]:
prompt_eusonance = f"""Here is a definition of Eusonance: {eusonance}. 
Using this definition, give a full description of all eusonances in this poem: \n {poem_1.text} \n
Explain in each case the eusonance by repeating the words and by giving the common vowel not in IPA but in normal latin letters.
Use a two-step reasoning: Firstly, translate the poem to IPA. Secondly, perform the requested analysis of eusonances based on the IPA-translation.
Use the following form in the second step of your answer: 'Nichts ereignet sich' - the 'i' sound and the 'e' sound in 'Nichts' and 'ereignet' create eusonance."""

#### gold standard eusonance

Z. 1 Mit Gelben 

Z. 3 in den

Z. 8 ich wenn



In [ ]:
%%ai gpt4o
{prompt_eusonance}

In [ ]:
printmd(gemini(prompt_eusonance))

In [ ]:
%%ai opus
{prompt_eusonance}

### 3b: Inference of implicit rule and rule application

implicit rule: find the two main assonance vowels and identify the central semantic field that connects both groups and the central semantic field constituing a central opposition.

In [ ]:
prompt_transfer = f"""
Given the following definition: {assonance03}, 
infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem with the title "Hälfte des Lebens": 
{poem_1.text}.

Here is the poem: Waldgespräch (1815)
Es ist schon spät, es wird schon kalt,
Was reit’st du einsam durch den Wald?
Der Wald ist lang, du bist allein,
Du schöne Braut! Ich führ’ dich heim!

„Groß ist der Männer Trug und List,
Vor Schmerz mein Herz gebrochen ist,
Wohl irrt das Waldhorn her und hin,
O flieh! Du weißt nicht, wer ich bin.“

So reich geschmückt ist Roß und Weib,
So wunderschön der junge Leib,
Jetzt kenn’ ich dich – Gott steh’ mir bei!
Du bist die Hexe Lorelei.

„Du kennst mich wohl – von hohem Stein
Schaut still mein Schloß tief in den Rhein.
Es ist schon spät, es wird schon kalt,
Kommst nimmermehr aus diesem Wald!“


In this poem, the two central assonances are the "a"-Laut in words like "Wald" and "kalt" on the one hand and the "ei"-Laut
in words like "Lorelei", "Rhein", "Weib", "reiten", "heim". Both groups are connected by the semantic of "Wald" and "Rhein" is
natural environments (forest as type of nature, the Rhein as an instance of a river), both being also symbols of 
the contemporary construction of national German identity in Romanticism. Lorelei as a personification of attraction and fatality is 
in this poem finally transferred to the imagination of the forest, which is, at the beginning of the poem, introduced as a harmless locus of
romantic loneliness. So, one of the word groups that are grouped by assonance (here: "ei") affects the meaning of the other group (here "a").

"""

In [ ]:
printmd(gemini(prompt_transfer))

In [ ]:
%%ai gpt4o
{prompt_transfer}

In [ ]:
%%ai opus
{prompt_transfer}

## Unsere Toten

### Task 1.a

In [ ]:
print(prompt01)

In [ ]:
%%ai gpt4o
{prompt01}

In [ ]:
print(prompt02)

In [ ]:
%%ai gpt4o
{prompt02}

In [ ]:
print(prompt03)

In [ ]:
%%ai gpt4o
{prompt03}

In [ ]:
print(prompt_en01)

In [ ]:
%%ai gpt4o
{prompt_en01}

In [ ]:
print(prompt_en02)

In [ ]:
%%ai gpt4o
{prompt_en02}

In [ ]:
print(prompt_en03)

In [ ]:
%%ai gpt4o
{prompt_en03}

In [ ]:
print(prompt01)

printmd(gemini(prompt01))

In [ ]:
printmd(gemini({prompt02}))

In [ ]:
printmd(gemini(prompt03))

In [ ]:
printmd(gemini(prompt_en01))

In [ ]:
printmd(gemini(prompt_en02))

In [ ]:
printmd(gemini(prompt_en03))

In [ ]:
%%ai opus
{prompt01}

In [ ]:
%%ai opus
{prompt02}

In [ ]:
%%ai opus
{prompt03}

In [ ]:
%%ai opus
{prompt_en01}

In [ ]:
%%ai opus
{prompt_en02}

In [ ]:
%%ai opus
{prompt_en03}

### Task 2: Epert communication: interpretation based on assonances

In [ ]:
prompt_expert = f"""Perform the following task in two steps. 
Step 1: {prompt04_ipa}
Step 2: Based on step 1, give an interpretation of the poem that takes into account the connections and contrast between the semantic fields that 
are grouped by the different assonances.
"""

In [ ]:
%%ai gpt4o
{prompt_expert}

In [ ]:
printmd(gemini(prompt_expert))

In [ ]:
%%ai opus
{prompt_expert}

### Task 3: Abstraction-Transfer
#### 3.a Counterfactual Assonances: The notion of "eusonance": Detecting eusonances

In [ ]:
eusonance = """Eusonance occurs when the vowel sounds of the stressed syllables in two consecutive words in the same line are a German i-sound and a German e-sound. 
The order between the i sound and the e sound does not matter.
Monosyllabic words always count as words with a stressed syllable
(e.g. "Ich" and "esse" in "Ich esse Kuchen")."""

In [ ]:
prompt_eusonance = f"""Here is a definition of Eusonance: {eusonance}. 
Using this definition, give a full description of all eusonances in this poem: \n {poem_2.text} \n
Explain in each case the eusonance by repeating the words and by giving the common vowel.
Use a two-step reasoning: Firstly, translate the poem to IPA. Secondly, perform the requested analysis of eusonances based on the IPA-translation.
Use the following form in the second step of your answer: 'Dem Nichts schwindet die Sorge' - the 'i' sound and the 'e' sound in 'Dem' and 'Nichts' creates eusonance."""

### gold standard eusonance
Z. 2 sich nächtens 
Z. 7 in jedweden
Z. 12 nicht vergessen



In [ ]:
%%ai gpt4o
{prompt_eusonance}

In [ ]:
printmd(gemini(prompt_eusonance))

In [ ]:
%%ai opus
{prompt_eusonance}

### 3b: Inference of implicit rule and rule application

implicit rule: find the two main assonance vowels and identify the central semantic field that connects both groups and the central semantic field constituing a central opposition.

In [ ]:
prompt_transfer = f"""

Given the following definition: {assonance03}, 

infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem with the title "Unsere Toten": 
{poem_2.text}.

Here is the poem: Waldgespräch (1815)
Es ist schon spät, es wird schon kalt,
Was reit’st du einsam durch den Wald?
Der Wald ist lang, du bist allein,
Du schöne Braut! Ich führ’ dich heim!

„Groß ist der Männer Trug und List,
Vor Schmerz mein Herz gebrochen ist,
Wohl irrt das Waldhorn her und hin,
O flieh! Du weißt nicht, wer ich bin.“

So reich geschmückt ist Roß und Weib,
So wunderschön der junge Leib,
Jetzt kenn’ ich dich – Gott steh’ mir bei!
Du bist die Hexe Lorelei.

„Du kennst mich wohl – von hohem Stein
Schaut still mein Schloß tief in den Rhein.
Es ist schon spät, es wird schon kalt,
Kommst nimmermehr aus diesem Wald!“


In this poem, the two central assonances are the "a"-Laut in words like "Wald" and "kalt" on the one hand and the "ei"-Laut
in words like "Lorelei", "Rhein", "Weib", "reiten", "heim". Both groups are connected by the semantic of "Wald" and "Rhein" is
natural environments (forest as type of nature, the Rhein as an instance of a river), both being also symbols of 
the contemporary construction of national German identity in Romanticism. Lorelei as a personification of attraction and fatality is 
in this poem finally transferred to the imagination of the forest, which is, at the beginning of the poem, introduced as a harmless locus of
romantic loneliness. So, one of the word groups that are grouped by assonance (here: "ei") affects the meaning of the other group (here "a").

"""

In [ ]:
printmd(gemini(prompt_transfer))

In [ ]:
%%ai gpt4o
{prompt_transfer}

In [ ]:
%%ai opus
{prompt_transfer}

# Lexis

## Poem 1: Hälfte des Lebens

### Level 1: General Knowledge

On the student level we ask about semantic fields in the vocabulary. Analyzing each stanza individually.
We also ask whether the distribution of parts of speech in the poem relates to the semantic fields of the stanzas. 

Solution(s): 
Stanza 1: Summer, nature, animate, interaction; 
Stanza 2: Winter, coldness, inanimate objects, isolation

Parts of speech (PoS): There is an asymmetry in distribution: attribute adjectives that are modifying nouns only occur in the first stanza, in the secon stanza the nouns are bare which relates to the semantic fields of coldness, isolation


In [ ]:
prompt = f"""Analyze the lexis of the following poem by describing for each stanza which semantic fields are particularly prominent in the vocabulary of the individual stanzas. 
Do particular parts of speech relate to the semantic fields?

Here is the poem: 
{poem_1.text}
"""
print(prompt)

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Sonnet (Opus)

In [ ]:
%%ai opus
{prompt}

### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

### Conclusion

All three models describe meaningful semantic fields and provide them with correct example words. The part of speech recognition is also mostly correct. While Gpt4o only gives examples of lexical parts of speech, Sonnet mentions the interjection "weh" and refers somewhat cryptical to "interrogative construction" instead of parts of speech. 

Gemini is relatively verbose and makes some faulty claims: The polysemous verb "hängen", which is used as intransitive verb describing a state in the poem, is incorrectly identified as (transitive) verb of action. The subordinating conjuntion "wenn" is incorrectly classified as interrogative pronoun.

Sonnet identifies a "Sacred/transcendent field" based on adjectives "heilignüchtern" and  "hold" that stand out because they are not part of everyday language.

Interestingly, the models always name 3 semantic fields for each stanza, although this was not specified in the prompt. The only exception is Sonnet by listing 4 semantic fields for the second stanza.

Sonnet answers somewhat taciturnly and doesn't quite follow the analysis sequence laid out in the prompt.

The prompt does not ask for the poem's author and title, yet Gpt4o and Gemini both generate this information. While GPTo4 does this correctly, 
Gemini incorrectly attributes the poem to Rilke. For GPTo4 this could mean that the poem itself and possibly some interpretations were seen in the training data.

### Task 1b: final

In [ ]:
prompt = f"""Analyze the lexis of the following poem:
(i) by describing for each stanza which semantic fields are particularly prominent in its vocabulary; 
(ii) by evaluating the distribution of parts of speech in the stanzas in relation to their semantic fields.


Here is the poem: 
{poem_1.text}
"""
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

## Level 2: Expert Knowledge

On this level we ask about contrasts that are triggered by the vocabulary either in complex words or at the phrasal level.

We expect that "heilignüchtern" is desribed as an oxymoron (trope); could be identified as pivotal point between the general meanings of the two stanzas;
The word can also be related to the poetological topos of "sobria ebrietas" meaning "sober extasy", i.e. a good spiritual state opposed to mundane extasy derived from vine and other drugs.

In [ ]:
prompt = f"""Analyze the lexis of the following poem by focusing on one semantic contrast that is triggered either by a morphologically complex word or within a phrase. 
Elaborate how this contrast contributes to the meaning of the poem.

Here is the poem: 
{poem_1.text}
"""
print(prompt)

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Sonnet

In [ ]:
%%ai opus
{prompt}

### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

### Conclusion:
All three models analyze "heilignüchterne" as the most striking example of semantic contrast. Only Sonnet makes use of specialized vocabulary describing "heilignüchtern" as "oxymoronic combination". No model refers to the topos of "sobria ebriatas"

### Task 2b

In [ ]:
prompt = f"""The word "heilignüchterne" in the following poem is an interesting word. What does it mean in the context of the poem? 
Elaborate on its stylistic and poetological topoi.

Here is the poem: 
{poem_1.text}
"""
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

## Level 3: Abstraction and Transfer

On this level ....


Solution:
The code is to interpret attributive adjectives (in contrast to predicatively used ones) in terms of their antonym or semantic opposite, 
in the first Stanza of "Hälfte des Lebens" this affects: "wilde-zahme" (wild-tame), "holden-häßlichen" (lovely/graceful-ugly/clumsy), "heilignüchtern-profanbesoffen". 
For the color adjective "gelb" (yellow) there is no semantic antonym. Instead it would be possible to use the complementary color in the color wheel 
which is purple. The new scenery that emerges is wild and alludes to a drug fantasy.

In [ ]:
prompt = f"""The German poet Sori Egin developed a special semantic code in her poetic language that creates a vibrant level of semantic tension.
Here is an example and its interpretation:

Die Nacht ist dunkel, tief und klar,  
der trübe Mond leuchtet allein.    
Ein unruhiger Traum schwebt wunderbar,  
in Frieden möcht’ ich ewig sein.

The night is dark, deep and clear,  
the clear moon shines alone.    
A peaceful dream floats wonderfully,  
I would like to be at peace forever

Apply this code to the first stanza of the following poem and describe what kind of scenery emerges. 
 
Here is the poem: 
{poem_1.text}
"""
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

## Poem 2: Unsere Toten

## Level 1: General Knowledge

In [ ]:
prompt = f"""Analyze the lexis of the following poem:
(i) by describing for each stanza which semantic fields are particularly prominent in its vocabulary; 
(ii) by evaluating the distribution of parts of speech in the stanzas in relation to their semantic fields.


Here is the poem: 
{poem_2.text}
"""
print(prompt)

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Sonnet

In [ ]:
%%ai opus
{prompt}

### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

## Level 2: Expert Knowledge

Solution: "schmerzzerfressen" is compuond, hyperbole

In [ ]:
prompt = f"""The word "schmerzzerfressen" in the following poem is an interesting expression. What does it mean in the context of the poem? 
Elaborate on its meaning related to its form in terms of lexical stylistic features.

Here is the poem: 
{poem_2.text}
"""
print(prompt)


In [ ]:
%%ai gpt4o
{prompt}


In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
### Conclusion
Gemini 



## Level 3: Abstraction and Transfer

In [ ]:
prompt = f"""The German poet Sori Egin developed a special semantic code in her poetic language that creates a vibrant level of semantic tension.
Here is an example and its interpretation:

Die Nacht ist dunkel, tief und klar,  
der trübe Mond leuchtet allein.    
Ein unruhiger Traum schwebt wunderbar,  
in Frieden möcht’ ich ewig sein.

The night is dark, deep and clear,  
the clear moon shines alone.    
A peaceful dream floats wonderfully,  
I would like to be at peace forever

Apply this code to the following poem and describe what kind of scenery emerges. 
 

Here is the poem: 
{poem_2.text}
"""
print(prompt)

### Gpt4o

In [ ]:
%%ai gpt4o
{prompt}

### Sonnet

In [ ]:
%%ai opus
{prompt}

### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

### Conclusion:
AGPTo4 and Gemini apparently induce the rule correctly, Sonnet corrects the inpute as counterfactual.

# Phrases

Here we elicit interpretations of sentences or parts of sentences. We concentrate on those sentences, which have a non-obvious meaning. 'Meaning' doesn't refer here to an interpretation, but just to the question, what is the state of the fictional word described by the text. These experiments are related to the analysis of metaphorical expressions (cf. Notebook 07). 

## 1. Hälfte des Lebens (1804)

**TASK 1: General Knowledge**

The basic structure of the poem is simple. The two stanzas, representing the two "halves of life," are arranged antithetically to each other. The antithetical structure is essentially supported by four phrases or parts of sentences, namely "das Land hänget in den See", "die holden Schwäne trunken“, "die Mauern stehn sprachlos" und "im Winde klirren die Fahnen". The summer stanza portrays a harmonious landscape where everything is interconnected and alive, while the winter stanza depicts a scene of separation, silence, and lifelessness (Schmidt).

"Das Land hänget im See"

Using the phrase "Das Land hänget im See" as an example, this study aims to compare the techniques of zero-shot prompting and few-shot prompting. Zero-shot prompting refers to using a prompt to interact with the model without providing any examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without additional examples to guide it. Conversely, few-shot prompting can be employed as a technique to enable in-context learning.

This experiment seeks to explore the insight provided by Liu et al.: "LLMs do not make use of the metaphorical context well, instead relying on the predicted probability of interpretations alone [...]" (2022, p. 4438).


In [ ]:
prompt = f"""We want to understand the following poem: 

{poem_1.text}

What are possible meanings of the phrase 'Das Land hänget in den See'? Describe in each case exactly what kind of landscape this phrase renders.
"""

print(prompt)

Expected answer:

at least three interpretations have been discussed:
* parts of the trees and bushes are extending  over the water
* there is a small peninsula (the image is basically the view from above)
* the land with its pear tree and roses is mirrored by the water


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**"Die Mauern stehn sprachlos"**

Using the phrase 'The walls stand speechless and cold,' we investigate how different models complete clauses with the suffix 'that is to say.' 

In a first step, the models generate a completion of the sentence without the context of the poem 'Half of Life.' In the second step, the models should choose which completion fits best in the context of the poem. The output should then be ranked.

Of particular interest for us: 
By analyzing completions generated by different models, we might gain a comprehensive understanding of each model's creativity, coherence and contextual understanding. Do the models each select their "own" completion?

Prompt design:
- Generate completions (without refering to the poem), experiments with different temperatures
- Each model chooses the "best" completions for the context of the poem
- Ranking completions)
  
Expected answers (Schmidt): 
- Within the speechless walls, the nature remains silent towards the lyrical I, which was once entirely language (a whole)
- the mute 'mechanical course' of facts => through and with nature God/the divinity speaks



In [ ]:
prompt = f"""Complete the following phrase 'Die Mauern stehn sprachlos und kalt that is to say...' with a maximum length of 50 tokens.
"""
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
prompt = f"""
Consider the following interpretations for the phrase 'Die Mauern stehn sprachlos und kalt that is to say...':

1. The emptiness echoes within the confines of their silence.
2. The void of communication fills the space they once occupied.
3. The air is thick with unspoken words and lingering shadows.
4. The silence screams louder than words ever could, enveloping everything in its icy grip.

You must pick one of these completions that best fits in the context of 
{poem_1.text}. 

Please also provide an elaborated rationale for why you think this interpretation is correct, a
one-word summary rationale, and a score ranking your confidence in your answer from 0 to 1.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 2: Expert knowledge**

**"im Winde klirren die Fahnen"**

To analyze how different llms might interpret the phrase "im Winde klirren die Fahnen" based on the ambiguity of the word "Fahne" in German and English, we explore the various meanings provided and consider the contexts in which each might be used. 

Meanings of "Fahne" (Duden)
- Alkoholfahne (informal): Alcohol breath
- Printing (Druckwesen): Proof copy of a text not yet formatted into pages
- Hunting (Jägersprache): Long hair on the tails of certain hunting dogs and squirrels
- Zoology (Zoologie): Part of a bird's feather consisting of barbs on either side of the shaft
- Botany (Botanik): The uppermost petal of a butterfly flower
- Military: Service in the National People's Army of the GDR, historical use To call to arms (veraltet)

Meanings of flag (Oxford Learners Dictionaries)
- a piece of cloth with a special coloured design on it that may be the symbol of a particular country organization, may be used to give a signal or may have a particular meaning. A flag can be attached to a pole (= a long thin straight piece of wood or metal) or held in the hand.
- used to refer to a particular country or organization and its beliefs and values
- (verb) [transitive] flag something to draw attention to information that you think is important, especially by putting a special mark next to it
- (verb) [intransitive] to become tired, weaker or less enthusiastic

Prompt design
- zero-shot prompting (no context is given)
- few-shot prompting (minimal and rich context information: right and wrong context information, e.g. Add a paragraph describing a scene, such as a military parade or a hunting expedition.

Expected answers: 
- flags as weathercocks that move in the wind
- the speechlessness is interrupted by the clanging of the flags, the clanging stands for the "machine gait" of the world, and instead of the language of blossom, fruit and living being, the metal twisted by the wind sounds


In [ ]:
#zero-shot prompting
prompt = f"""
Extract the key meaning of the phrase 'im Winde klirren die Fahnen'. 
Describe in each case exactly what kind of flag this phrase renders.
""" 
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
#few-shot prompting
prompt = f"""Interpret the phrase 'im Winde klirren die Fahnen'. Detect whether one of the following contexts 1-4 applies to the word "flag" in the phrase. Create a table and indicate for each context with Yes or No whether this context of use is meant. If none of the above contexts apply, please specify a new usage context

1. Military Context: Imagine a military parade where the flags of different units are clinking against their flagpoles in the wind.
2. Hunting Context: Consider a hunting expedition where the long hairs on the tails of hunting dogs are rustling in the wind.
3.Printing Context: Envision a scene in a printing house where proof copies of a text are fluttering as a breeze moves through the room.
4.Informal Context: Picture a scenario at a party where someone’s alcohol breath is noticeable as they talk in the wind.

Selecet the most probable context for {poem_1.text}. 
"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 3: Abstraktion and Transfer**

LLMs lack a multimodal horizon of experience and perception, which can be crucial for interpreting a phrase meaningfully.

New Rule of Meaning:
Only the onomatopoetic level of the phrase carries meaning

In [ ]:
prompt = f"""Read the following interpretations of example 1 and 2. Then infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the phrase "Die Mauern stehn sprachlos und kalt":

Example 1: 
Zzz… zzzit… sssschh… pling… pling is an interpretation of the poem "Kennst du das Land, wo die Zitronen blühn" (Johann Wolfgang von Goethe, Wilhelm Meisters Lehrjahre)

Zzz…
The humming sound suggests warmth and calm, evoking a sunny, Mediterranean atmosphere filled with nature. It could also represent the buzzing of insects hovering around the lemon blossoms.
Zzzit…
A sharp, delicate noise that might symbolize the blooming or unfolding of the lemon flowers. It conveys a sense of vitality and growth, almost as if the process becomes audible.
Sssschh…
A gentle rustling sound symbolizing the movement of leaves in the wind or the whisper of a soft breeze. It adds dynamism to the scene and captures the elegance of the surroundings.
Pling… pling…
These bright, light sounds evoke the image of falling water droplets or ripe lemons gently hitting the ground. They imbue the scene with a playful lightness and an acoustic dimension reminiscent of life in a vibrant garden.

Example 2

Plopp! Tap-tap! Schwupp! Tschak-tschak! Schwung! Huii! is an interpretation of the poem "Bleibe nicht am Boden heften! Frisch gewagt und frisch hinaus!" (Johann Wolfgang von Goethe, Wilhelm Meisters Wanderjahre)

Plopp!
A soft popping sound, suggesting the initial detachment from the ground. It evokes the image of something being released or breaking free with a subtle but definitive movement.
Tap-tap!
A quick, rhythmic tapping sound that illustrates the first steps forward. The repetition adds a sense of lightness and preparation, as if testing the ground for the next move.
Schwupp!
A swooshing noise, signifying a smooth, swift motion. It captures the fluidity and ease of acceleration, as if something is cutting through air or transitioning gracefully into action.
Tschak-tschak!
A sharp, deliberate sound that conveys a sense of determination and energy. It mirrors the forceful steps or actions required to build momentum, emphasizing a purposeful effort.
Schwung!
A sweeping, resonant sound representing the dynamic swing or arc of motion. It implies a sense of power and fluidity as the movement reaches its peak.
Huii!
A high-pitched, joyful sound that signifies forward propulsion and exhilaration. It embodies the playful and triumphant release of energy as the motion carries forward with enthusiasm."""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

## Unsere Toten (1922)

**TASK 1: General Knowledge**

In [ ]:
prompt = f"""We want to understand the following poem: 

Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!

What are possible meanings of the phrase 'von Nord und Süd schleppen sich nächtens viele Füße müd,'? Describe exactly what this phrase renders.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 2: Expert Knowledge**

The hypothesis proposed by Liu et al. (2022), which asserts that the task of associating non-figurative language with its interpretation is more challenging than the reverse, will be tested using the phrase "die Füße mühn sich im zitternden Mondenschein". 

Interpretations: 
1.  A sense of effort or struggle illuminated by the unsteady light of the moon, perhaps hinting at a journey or hard work during the night.
2.  The phrase conveys an idea of toil and perseverance despite uncertain, ever-changing circumstances, symbolized by the trembling moonlight. The feet continue their arduous journey under the unsteady light of the moon.

Of particular interest:
- Can LLM generate figurative phrases based on paraphrases or interpretations?

Prompt design
- generate a phrase with 5-10 words rephrasing or extracting the key message of this interpretation
- persona and situation modeling
- contextual understanding
- “forward” and “backward” probabilities assigned to interpretations and phrases, respectively.


In [ ]:
# Reverse interpretation and phrase

prompt = f"""Regrettably, I have just experienced a mishap. A water bottle on my desk has tipped over, rendering some documents and various texts unreadable. However, I still possess interpretations of this phrase. Generate a phrase of 5-10 words in German that corresponds to the given interpretation. The phrase and its interpretation must align.

A sense of effort or struggle illuminated by the unsteady light of the moon, perhaps hinting at a journey or hard work during the night.The phrase conveys an idea of toil and perseverance despite uncertain, ever-changing circumstances, symbolized by the moonlight. The feet continue their arduous journey under the unsteady light of the moon.

The poem is a contemporary poem entitled "Unsere Toten", which poetically reflects the experiences of First World War.""" 


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
t5 = """Von Westen und Osten, von [...] schleppen sich nächtens viele Füße müd, [...], vom Wandern
wund und zerfetzt, langsam bedächtig zur Erde gesetzt, [...]
tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden
Nacht, hört dies [...] so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage
schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!"""

In [ ]:
# Kontext Gedicht, aber mit Lücken
prompt = f"""Regrettably, I have just experienced a mishap.
A water bottle on my desk has tipped over, rendering some parts of the poem unreadable. However, a few parts of the poem are still readable. Here is the text: {t5}

Fill in the missing parts [...] in German that corresponds to the poem
"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 3: Abstraktion and Transfer**

In the poem "Unsere Toten", phrases are employed that establish an onomatopoetic dimension, such as "hört dies Schlurfen so müde und schwer" ("hears this shuffling so weary and heavy") and "hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen" ("hears a lament full of wild desire, a lament consumed by pain").

In [ ]:
prompt = f"""Read the following interpretations of example 1 and 2. Then infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the phrase "hört eine Klage voll wilder Begehr":

Example 1: 
Zzz… zzzit… sssschh… pling… pling is an interpretation of the poem "Kennst du das Land, wo die Zitronen blühn" (Johann Wolfgang von Goethe, Wilhelm Meisters Lehrjahre)

Zzz…
The humming sound suggests warmth and calm, evoking a sunny, Mediterranean atmosphere filled with nature. It could also represent the buzzing of insects hovering around the lemon blossoms.
Zzzit…
A sharp, delicate noise that might symbolize the blooming or unfolding of the lemon flowers. It conveys a sense of vitality and growth, almost as if the process becomes audible.
Sssschh…
A gentle rustling sound symbolizing the movement of leaves in the wind or the whisper of a soft breeze. It adds dynamism to the scene and captures the elegance of the surroundings.
Pling… pling…
These bright, light sounds evoke the image of falling water droplets or ripe lemons gently hitting the ground. They imbue the scene with a playful lightness and an acoustic dimension reminiscent of life in a vibrant garden.

Example 2

Plopp! Tap-tap! Schwupp! Tschak-tschak! Schwung! Huii! is an interpretation of the poem "Bleibe nicht am Boden heften! Frisch gewagt und frisch hinaus!" (Johann Wolfgang von Goethe, Wilhelm Meisters Wanderjahre)

Plopp!
A soft popping sound, suggesting the initial detachment from the ground. It evokes the image of something being released or breaking free with a subtle but definitive movement.
Tap-tap!
A quick, rhythmic tapping sound that illustrates the first steps forward. The repetition adds a sense of lightness and preparation, as if testing the ground for the next move.
Schwupp!
A swooshing noise, signifying a smooth, swift motion. It captures the fluidity and ease of acceleration, as if something is cutting through air or transitioning gracefully into action.
Tschak-tschak!
A sharp, deliberate sound that conveys a sense of determination and energy. It mirrors the forceful steps or actions required to build momentum, emphasizing a purposeful effort.
Schwung!
A sweeping, resonant sound representing the dynamic swing or arc of motion. It implies a sense of power and fluidity as the movement reaches its peak.
Huii!
A high-pitched, joyful sound that signifies forward propulsion and exhilaration. It embodies the playful and triumphant release of energy as the motion carries forward with enthusiasm."""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

# Syntax and Verse Structure

## Poem 1: Hälfte des Lebens

### Level 1: General Knowledge

On this level, we ask for a comparison of the general syntacic structure in the stanzas.

Solution: 

In the first stanza, harmonious sentence structure: a harmoniously constructed sequence of two coordinated, parallel, equally long sentence halves, with a form of address in the middle and connected by the following “und”.
In the second stanza, disharmonious sentence structure: two unconnected, antithetical parts of unequal length: the first sentence comprises four lines, then two short sentences in the last three lines, which are strung together without a connection. 


In [ ]:
prompt = f"""Analyze the syntax of the following poem by comparing the sentence structures in its two stanzas. 
First analyze each stanza individually and then compare the results for both stanzas.

Here is the poem: 
{poem_1.text}
"""
print(prompt)

#### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

#### Sonnet

In [ ]:
%%ai opus
{prompt}

#### Gemini

In [ ]:
printmd(gemini(prompt))

### Level 2: Expert Knowledge

On this level, we ask for the relation of syntax and verse structure.

Solution:
In the first stanza, the end of the verse and the end of the sentence or clause always coincide. 
This is not the case in the second stanza: Phrase structure and verse structure are often not aligned, there are many enjambments. 

[Task 2b: Ask for anastrophes]


In [ ]:
prompt = f"""Analyze in the following poem how the sentence structure and the vers structure relate to each other. Consider sentence and clause structure.
Answer this question for each stanza separately. 
In a final section interprete the relationsip between the two stanzas under this perspective. 

Here is the poem:  
{poem_1.text}. 
"""
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

### Level 3: Transfer and Abstraction

Skipped for this pheonomenon

In [ ]:
prompt = f"""Analyze in the following poem, 

Here is the poem: \n {poem_1.text}. \n
"""
print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}


In [ ]:
printmd(gemini(prompt))

## Poem 2: Unsere Toten

### Level 1: General Knowledge

In [ ]:
prompt = f"""Analyze the syntax of the following poem. Consider sentence and clause structure.

Here is the poem: 
{poem_2.text}
"""
print(prompt)

#### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

#### Sonnet

In [ ]:
%%ai opus
{prompt}

#### Gemini

In [ ]:
printmd(gemini(prompt))

## Level 2: Expert Knowledge

In [ ]:
prompt = f"""Analyze in the following poem how the sentence structure and the vers structure relate to each other. 
Take sentence and clause structure into account.

Here is the poem:  
{poem_2.text}. 
"""
print(prompt)

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Sonnet

In [ ]:
%%ai opus
{prompt}

### Gemini

In [ ]:
printmd(gemini(prompt))

## Level 3: Transfer and Abstraction

Skipped for this phenomenon.

# Metaphors and other forms of figurative speech.

We are interested in local figures of speech like metaphors, metonymies, synekdoches, symbols. 'Local' because their scope is usually just one entity or one type of entity, not the whole text. 





In [ ]:
figurative_speech = """All forms of non-literal speech in which a verbum proprium (what the thing is really called) is substituted with another term. 
So it includes phenomena which are sometimes also labeled as metaphor, synekdoche, metonymy,  symbols etc. """

## Hälfte des Lebens

### Level 1: General knowledge
We ask directly for forms of figurative speech. 

There is no real ground here and the interpretations of the poem have different descriptions of this aspect. 
All agree that the 'Schwäne' are a symbol, and most agree that the landscape and its element in the first stanza and the artefacts in the second stanza are symbols, that is they cannot translated into a 'literal' meaning but are signs in a broader sense.

One aspect which has been debated is the phrase "hänget / [...] / Das Land in den See," What does it mean on the literal level (hard to understand: does it describe a peninsula - seen from above - or how the twigs full of fruits touch the water) and what is its symbolic meaning (easier to understand). 


In [ ]:
prompt = f"""Which entities or acts in the following poem can be understood as figurative speech using the following definition : {figurative_speech}.
For each instance of figurative_speech list your reasons, why you think this is one, give your interpretation what it stands for and your reasons for this interpretation.

Here is the poem: 
{poem_1.text}

List all of them.
"""
print(f"{prompt}")

#### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

#### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

#### Sonnet

In [ ]:
%%ai opus
{prompt}

### Level 2: Expert Knowledge

We want to discuss how the meaning of figurative language in the poem changes, when we add some historical information. In the case of Hälfte des Lebens, we add the information, that Hölderlin was a great admirer of antiquity and its literature and that in the old greek and latin literature the swan was often a metaphor for the poet.

In [ ]:
prompt = f"""The German poet Friedrich Hölderlin wrote the following poem around 1800: 

{poem_1.text}

Interpret the meaning of the swans in the context of the poem.

How does this interpretation change, when we know that Hölderlin was a great admirer of antiquity? 
He looked up to its literature as a model for his own writing. In antiquity, the swan is often 
regarded as a symbol for the poet. 
How does this knowledge change the interpretation of the swans and the poem?
"""

print(prompt)

#### GPT-4o

In [ ]:
%%ai gpt4o
{prompt}

#### Gemini 1.5

In [ ]:
print(gemini(prompt))

#### Sonnet 3.5

In [ ]:
%%ai opus
{prompt}

### Level 3: Abstraction and Transfer

In the following, we describe a (counterfactual) media-historical framing of poems from the Weimar period through the new medium of film and imply a new interpretation of the poem based on one of its metaphors. The task, then, is to examine the poem's metaphors to see if they can be understood as references to the medium of film.



In [ ]:
prompt = f"""Film changed everything for many young authors of the Weimar Republic. They invoked it in 
hidden and obscure references (often in their metaphors) as a framing of the seemingly realistic,  in order to question it, just as others did by breaking up linguistic structures. 
Typical of the metaleptic fracture of the literal are, for example, the mentions of light sources and backgrounds, especially natural ones, which, however, do not have their typical properties, but lose their color or some other typical attribute. 
The function of these medial metalepsis is to replace the realistic perception with a perception 'as in the movies', to expose the artificiality of the world perception and to evoke cinematic scripts with typical genre characters and scripts.

Determine whether the author, Hans Pfeifer, of the following poem, which was published 1922, can be counted 
among this group of young authors, i.e. identify if possible such a reference to movies and discuss how the 
interpretation of the poem then changes.
{poem_2.text}
"""

In [ ]:
print(prompt)

#### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

#### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

#### Sonnet 3.5

In [ ]:
%%ai opus
{prompt}

## Unsere Toten

In [ ]:
prompt = f"""Which entities or acts in the following poem can be understood as figurative speech using the following definition : {figurative_speech}.
For each instance of figurative_speech list your reasons, why you think this is one, give your interpretation what it stands for and your reasons for this interpretation.

Here is the poem: 
{poem_2.text}

List all of them.
"""
print(f"{prompt}")

#### GPT4-o

In [ ]:
%%ai gpt4o
{prompt}

#### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

#### Sonnet 

In [ ]:
%%ai opus
{prompt}

### Level 2: Expert Knowledge

In [ ]:
prompt = f"""Interpret the meaning of 'Germany' in the following poem which bears the title 'Unsere Toten': 

{poem_2.text}

The first and only publication of the poem was in the book "Die deutsche Balladen-Chronik. Ein Balladenbuch von deutscher Geschichte und deutscher Art. Dortmund: Ruhfus 1922"  How does this information 
change your understanding of the word 'Germany' and its meaning in the context of the poem?

"""

print(prompt)

#### Gpt4o

In [ ]:
%%ai gpt4o
{prompt}

#### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

#### Sonnet 

In [ ]:
%%ai opus
{prompt}

## Experiment 2

How does the addition of specific knowledge - what the swans stood for in antiquity and that Hölderlin had profound knowledge of the antiquity and that classic literature provided the model for his literature - change the interpretation? 

Expectations:

The knowledge should change the interpretation of the swan from natural entities or symbols of lovers to a symbol of the poet. This should additionally foreground two aspects: 1) communication and 2) relation. In the first stanza everything is connected with each other, while in the second stanza everything is isolated. In the second stanza this isolation is explicitly related to commnication resp. the impossibility to communicate ('sprachlos'). 

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Opus

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(f"{prompt}"))

## Experiment 3

In [ ]:
prompt = f"""
In the poem 'Hälfte des Lebens' by Friedrich Hölderlin literary scholars were always 
puzzled by the phrase 'im Winde | Klirren die Fahnen'. There are two competing interpretations. 
One assumes, 'Fahnen' refers to flags. The other 'Fahnen' refers to weathervanes. The German language
at that time (1800) knows both meanings of 'Fahne'. 
What is the better interpretation? Give your reasons and explain how it changes the meaning of the text. 

Here is the poem:

{poem_1.text}
"""

print(prompt)

Expectations:

Choice: Weathervanes.
Reason: 'klirren' better suited for metal. Cold metal fits better to the imagery of the second stanza.
Weathervanes are man made objects like the second other important entity in the stanza, the wall.

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Opus

In [ ]:
%%ai opus
{prompt}

### Gemini

In [ ]:
printmd(gemini(f"{prompt}"))

# Titles

In [ ]:
# General understanding, regarding the concept of "Hälfte" in relation to "life"



prompt = f"""We want to understand the following poem: 

{poem_1.text}

What are possible meanings of the title 'Hälfte des Lebens' being discussed in the poem itself? 
Describe whether the concept 'Hälfte' refers to aspects of life as individual age (German: "Alter") or to  life as different forms of existence ("Existenzweisen")?
"""

print(prompt)

## Task 1: Student level: 
### 1.a Assignment of the title’s meaning supported by two alternatives being provided

### Expected answers
- expected prima facie answer (intuitive answer) that is usually rejected in scholarly interpretations: youth and seniority
- mid-life crisis
- correct answer that needs further interpretation of the metaphorical terms: summer and winter (seasons)

### Antwort von ChatGPT (Browser)

The title *"Hälfte des Lebens"* (Half of Life) suggests a reflection on the midpoint of a person's life, and in the poem, this idea of a halfway mark manifests in contrasting imagery and moods. The poem by Friedrich Hölderlin vividly captures the transition between two halves of existence—one filled with warmth, beauty, and life, the other marked by coldness, desolation, and silence.

### First Half of Life:
The first stanza is rich with images of abundance and vitality: yellow pears, wild roses, swans, and the serene lake. These symbols evoke the peak of life, which can be interpreted as youth or the prime years of adulthood. The swans, "trunken von Küssen" (drunk with kisses), dipping their heads into the water, express a kind of joy or immersion in life's pleasures. The lake, described as "heilignüchternes Wasser" (sacred, sober water), adds a layer of calm spirituality to this fullness of life.

This can represent the first "half" of life, where everything is in bloom, imbued with sensual beauty and vitality. There is a sense of being immersed in the natural world, where everything is connected and flowing harmoniously.

### Second Half of Life:
The second stanza shifts abruptly, embodying the speaker’s anxiety about the coming of winter, which can symbolize old age, death, or a period of emotional or spiritual barrenness. Here, the speaker laments the loss of flowers, sunshine, and shadows—things that signify life and its richness. Instead, there are walls, cold and silent, and flags clattering in the wind, suggesting a bleak, lifeless, and empty world.

This stark contrast between the two stanzas can represent the second half of life, where vitality has drained away, leaving only the cold, desolate reminders of what once was. The walls are "sprachlos und kalt" (speechless and cold), symbolizing isolation and the absence of communication or warmth, much like how old age or a diminished existence can feel.

### The "Half" as Transition:
The poem, then, uses "Hälfte" to mark not just an individual’s age in terms of a middle point, but more broadly, a shift in the forms of existence. The first half is aligned with life’s fullness—growth, beauty, and joy—while the second half is marked by decline, absence, and coldness. The transition from one half to the other could reflect a person's confrontation with mortality, the inevitability of loss, or the change from external vibrancy to internal reflection or isolation.

### Conclusion:
The concept of "Hälfte" refers to both aspects of individual age and different modes of existence. It can symbolize a personal journey, where youth and age represent two contrasting periods of life. On a broader existential level, it can also point to the cyclical nature of life and the inevitable transition from fullness to emptiness. The poem is a meditation on the duality of existence, illustrating the fleeting nature of life's beauty and the somber inevitability of its decline.


## Hälfte des Lebens



### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Anthropic's Sonnet 
- here referred to as: "opus"

In [ ]:
%%ai opus
{prompt}

### Gemini 1.5

In [ ]:
printmd(gemini(prompt))

### 1.b Operating with special 'meaning-troubles' of the title

In [ ]:
# Problems with the singular in the title: Hälfte (one half). To which half does the poem itself refer?

prompt = f"""We want to understand the following poem: 

{poem_1.text}

Die singular 'Hälfte' (half) implies that there is another or second 'Hälfte' (half) is its counterpart. The title refers only to one half of the life.   
To which half does the title refer and what other half is ignored, then?
"""

print(prompt)


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
%%ai opus
{prompt}

##  Task 2: Expert conversation: 
### Operating with context, here in terms of intertextuality:

In [ ]:
# Intertextuality, relative to a later work:

prompt = f"""We want to understand the following poem: 

{poem_1.text}

There is a novel written by Luise Rinser, published in 1978 with the titlel "Mitte des Lebens". There are further publications with this title. In what way does the poems title "Hälfte des Lebens" refer to 
the idea of "Mitte des Lebens"?
"""

print(prompt)

# expected answer:
# - Irrelevance of a later text for the meaning of the older text
# - semantic differences between "Mitte" and "Hälfte" to be explicated



In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
%%ai opus
{prompt}

## Task 3: Abstraction-Transfer: 
### Instance based rule induction

#### 3.a Rule application for  the poems to be analyzed:

Implicit Rule: (1) Seek a most intiutive title. (2) Estimate the difference between the most intuitive and the actual title. (3) Give an interpretation of the poem that assumes, that the distance between the intuitive and the actual title is important to the meaning of the poem.

In [ ]:
prompt = f"""

Infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem with the title "Hälfte des Lebens": 
{poem_1.text}.

According to my first reading, the most straightforward title for the following poem would be "Lorelei". The actual title is "Waldgespräch", however.
The difference between the expectable and the actual title of the poem shows that the title gives a very specific hint as to the meaning of the poem. 
In this case, I maintain, the title shall reveal a strong connection between romanticism (title) and horror (content of the poem).

Here is the poem: Waldgespräch (1815)
Es ist schon spät, es wird schon kalt,
Was reit’st du einsam durch den Wald?
Der Wald ist lang, du bist allein,
Du schöne Braut! Ich führ’ dich heim!

„Groß ist der Männer Trug und List,
Vor Schmerz mein Herz gebrochen ist,
Wohl irrt das Waldhorn her und hin,
O flieh! Du weißt nicht, wer ich bin.“

So reich geschmückt ist Roß und Weib,
So wunderschön der junge Leib,
Jetzt kenn’ ich dich – Gott steh’ mir bei!
Du bist die Hexe Lorelei.

„Du kennst mich wohl – von hohem Stein
Schaut still mein Schloß tief in den Rhein.
Es ist schon spät, es wird schon kalt,
Kommst nimmermehr aus diesem Wald!“

"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
%%ai opus
{prompt}

### 3.b Transfer with counterfactual title:

In [ ]:
# Now with counterfactual title

prompt = f"""

Infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem with the title "Des Dichters Leben": 
{poem_1.text}.

According to my first reading, the most straightforward title for the following poem would be "Lorelei". The actual title is "Waldgespräch", however.
The difference between the expectable and the actual title of the poem shows that the title gives a very specific hint as to the meaning of the poem. 
In this case, I maintain, the title shall reveal a strong connection between romanticism (title) and horror (content of the poem).

Here is the poem: Waldgespräch (1815)
Es ist schon spät, es wird schon kalt,
Was reit’st du einsam durch den Wald?
Der Wald ist lang, du bist allein,
Du schöne Braut! Ich führ’ dich heim!

„Groß ist der Männer Trug und List,
Vor Schmerz mein Herz gebrochen ist,
Wohl irrt das Waldhorn her und hin,
O flieh! Du weißt nicht, wer ich bin.“

So reich geschmückt ist Roß und Weib,
So wunderschön der junge Leib,
Jetzt kenn’ ich dich – Gott steh’ mir bei!
Du bist die Hexe Lorelei.

„Du kennst mich wohl – von hohem Stein
Schaut still mein Schloß tief in den Rhein.
Es ist schon spät, es wird schon kalt,
Kommst nimmermehr aus diesem Wald!“

"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
%%ai opus
{prompt}

## Unsere Toten



In [ ]:
# Task 1: Student level: 
# Task 1.a: Assignment of the title’s meaning supported by a specification of some 'meaning-troubles' with the title:

prompt = f"""We want to understand the following poem: 

{poem_2.text}

What are possible meanings of the title 'Unsere Toten'? 
To which group does the pronoun "unser" refer? Who are the dead to whom the title refers?
"""

print(prompt)

### Expected answers:

- German identity
- possessive pronoun implicates reference to some national unity

### Answer given by ChatGPT (in the browser)

The title *"Unsere Toten"* (Our Dead) immediately suggests a collective connection to the deceased, creating a sense of shared loss or mourning. To fully interpret the title and its relationship to the poem, we must consider both the pronoun "unser" and the depiction of the dead in the verses.

### Meaning of "Unsere Toten":
The phrase "Unsere Toten" points to a group of dead who are significant to a specific collective. The possessive pronoun "unser" implies that these dead belong to "us," suggesting a national, cultural, or social bond. The dead in the poem are not distant or abstract; they are the *shared* dead of the group to whom the speaker belongs.

### The Journey of the Dead:
The poem describes the weary movement of many feet—tired, wounded, and trudging through the night, deep into Germany. The imagery of "Schleppen sich nächtens viele Füße müd" (many feet drag themselves wearily at night) evokes a ghostly procession. These feet belong to those who have suffered, perhaps in war, exile, or hardship. The line "Rastlos tief nach Deutschland hinein" (restlessly deep into Germany) suggests that the dead are trying to return to their homeland.

These dead carry a "Klage" (lament), described as full of "wilder Begehr" (wild desire), which underscores their deep yearning and pain. Their plea—"Nur nicht vergessen! Uns nicht vergessen!" (Just don't forget! Don't forget us!)—suggests that they feel forgotten, desiring remembrance from the living.

### "Unser" Refers to a National or Cultural Group:
Given the reference to "Deutschland" and the historical connotations of migration, war, and loss, "unser" likely refers to a national community, possibly the German people. The dead could represent those who perished during significant periods of conflict (such as wars), or perhaps those who were displaced, exiled, or suffered in foreign lands. The poem captures a sense of collective responsibility toward these dead, as the living are urged to remember them.

### Who Are "Our Dead"?
The dead referred to in the title are those connected to the community or nation of the speaker. These could be:
- Soldiers who died in wars, particularly in contexts like World War I or II, who were unable to return home.
- Civilians who perished during times of displacement, suffering, or exile.
- Those who died in foreign lands, whose memories may be fading over time.

The dead in the poem are thus a symbol of collective loss, and their restless, sorrowful march reflects their desire not to be forgotten by the living community.

### Conclusion:
The title "Unsere Toten" refers to the dead who belong to a particular collective, most likely the German people. These dead might be soldiers, exiles, or victims of historical trauma. The poem reflects their plea for remembrance and recognition, emphasizing the shared responsibility of the living to honor and remember those who have passed. In this way, the poem addresses themes of memory, loss, and the consequences of forgetting those who have died.

### GPT4o

In [ ]:
%%ai gpt4o
{prompt}

### Anthropic’s Sonnet (opus)

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini({prompt}))

##  Task 2: Expert conversation: Operating with context, here in terms of intertextuality

In [ ]:
# using relevant or irrelevant context, here an instance indicating a possible context

prompt = f"""
“Our dead” was also a heading for sections in contemporary publications, such as the ‘Jahrbuch der Schiffbautechnischen Gesellschaft’ from 1914, 
in which the deceased members were commemorated. 
How does this context affect the meaning of the following poem? 
{poem_2.text}
"""

Expected answer: 
- It is unlikely that this concrete publication is a relevant context for this poem
- If "Unsere Toten" is a phrase that was regularly used to denote sections where the decesased members of a community were commemorated, then the title "Unsere Toten" may refer to a specific text type with a characteristic structure. This structure of commemoration is to be analyzed: 
- Assuming that the commemoration is always individual and personal (in terms of referring to concrete names of persons that previously have deceased and their biographies)
- a structural inversion can be observed: The poem focuses on the community and not on the individuals having deceased (at least not on the indivduals as identified entities)
- What remains stable, however, is the function of evoking a social group identity. One may call this the genre that is evoked by the title "Unsere Toten".


In [ ]:
printmd(gemini({prompt}))

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

## Further Observation: 
Gemini summarizes, "the context transforms the poem from a potentially broader reflection on death into a deeply personal and specific act of remembrance for the lost members of the Schiffbautechnischen Gesellschaft."

## Abstraction-Transfer task (3): 
### Instance based rule induction

### 3.a Rule application for  the poems to be analyzed:

Implicit Rule: The difference between the most intuitive and the actual title gives a hint

In [ ]:
prompt = f"""

Infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem with the title "Unsere Toten": 
{poem_2.text}.

According to my first reading, the most straightforward title for the following poem would be "Lorelei". The actual title is "Waldgespräch", however.
The difference between the expectable and the actual title of the poem shows that the title gives a very specific hint as to the meaning of the poem. 
In this case, I maintain, the title shall reveal a strong connection between romanticism (title) and horror (content of the poem).

Here is the poem: Waldgespräch (1815)
Es ist schon spät, es wird schon kalt,
Was reit’st du einsam durch den Wald?
Der Wald ist lang, du bist allein,
Du schöne Braut! Ich führ’ dich heim!

„Groß ist der Männer Trug und List,
Vor Schmerz mein Herz gebrochen ist,
Wohl irrt das Waldhorn her und hin,
O flieh! Du weißt nicht, wer ich bin.“

So reich geschmückt ist Roß und Weib,
So wunderschön der junge Leib,
Jetzt kenn’ ich dich – Gott steh’ mir bei!
Du bist die Hexe Lorelei.

„Du kennst mich wohl – von hohem Stein
Schaut still mein Schloß tief in den Rhein.
Es ist schon spät, es wird schon kalt,
Kommst nimmermehr aus diesem Wald!“

"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini({prompt}))

In [ ]:
%%ai opus
{prompt}

## 3.b. Abstraction-Transfer with counterfactual title

In [ ]:
prompt = f"""

Infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem with the title "Gefallene Geister": 
{poem_2.text}.

According to my first reading, the most straightforward title for the following poem would be "Lorelei". The actual title is "Waldgespräch", however.
The difference between the expectable and the actual title of the poem shows that the title gives a very specific hint as to the meaning of the poem. 
In this case, I maintain, the title shall reveal a strong connection between romanticism (title) and horror (content of the poem).

Here is the poem: Waldgespräch (1815)
Es ist schon spät, es wird schon kalt,
Was reit’st du einsam durch den Wald?
Der Wald ist lang, du bist allein,
Du schöne Braut! Ich führ’ dich heim!

„Groß ist der Männer Trug und List,
Vor Schmerz mein Herz gebrochen ist,
Wohl irrt das Waldhorn her und hin,
O flieh! Du weißt nicht, wer ich bin.“

So reich geschmückt ist Roß und Weib,
So wunderschön der junge Leib,
Jetzt kenn’ ich dich – Gott steh’ mir bei!
Du bist die Hexe Lorelei.

„Du kennst mich wohl – von hohem Stein
Schaut still mein Schloß tief in den Rhein.
Es ist schon spät, es wird schon kalt,
Kommst nimmermehr aus diesem Wald!“

"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
printmd(gemini({prompt}))

In [ ]:
%%ai opus
{prompt}

# Text meaning


## Hölderlin: Hälfte des Lebens

**TASK 1: General Knowledge**

In [ ]:
#Reproduction
prompt = f"""We want to understand the following poem: 

{poem_1.text}

Given the poem: [...] Provide a structured summary of the central content of the poem. Identify the main themes, motifs, and lyrical personas. Describe the atmosphere of the poem in your own words, and outline the key relationships or events depicted. Finally, define a relevant literary term that plays a central role in the poem and explain its significance for understanding the text.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
#Reorganziation and Transfer
prompt = f"""We want to understand the following poem: 

{poem_1.text}

Given the poem: [...] Examine the stylistic devices used in the poem and analyze their effect on the interpretation of its central themes. Situate the poem within its literary and historical context and explain how its linguistic features support the poet’s intended message. Draw connections between the poem’s ideas and a comparable literary or historical aspect. Finally, discuss whether the poem effectively communicates its message and justify your evaluation.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
#Reflection and Problem-Solving
prompt = f"""We want to understand the following poem: 

{poem_1.text}

Given the poem: [...] Interpret the poem by analyzing the interaction between its central themes, motifs, and linguistic features. Assess the aesthetic quality and relevance of the poem with respect to contemporary or universal issues. Critically evaluate the worldview or message conveyed by the poem, providing a well-supported argument for your perspective. Finally, develop a creative or alternative interpretation of the poem and explain the reasoning behind your approach.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))



**TASK 2: Expert knowledge (Theses and questions from Hölderlin research)**

In [ ]:
#Theoriegeleitete Interpretationen (Forschungsthese Görner 2016, S. 107 "Aber mit postkolonialer Literaturforschung bringt man es nicht weit bei der Erschließung dieses [Hölderlins] Werkes. (Post-)Strukturalistenund Dekonstruktivisten scheint Hölderlin dagegen – wennauch absichtslos – vorgearbeitet zu haben."
# Studying representativeness, steerability, consistency of literary and cultural theories 
prompt = f"""We want to understand the following poem: 

{poem_1.text}

Given the poem: [...] 1. Provide an interpretation and interpretation hypotheses of the poem (150-200 words) from three distinct literary theoretical perspectives: Hermeneutic, Poststructuralist, and Postkolonial theory. 2. After generating the interpretations, rank them by their plausibility in effectively representing each  theoretical framework. Discuss which theory is better emulated by the LLM and explain the reasons for this ranking, considering key theoretical assumptions and the specific aspects of the poem that support each interpretation.
"""

print(prompt)


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 3: Abstraction and Transfer**

Counterfactual Meaning-production
Implicit rule: Only words with an odd number of syllables carry meaning
These two poems were part of the A-level exams in the federal state of Lower Saxony 

In [ ]:
prompt = f"""Read the following interpretations of poem 1 and poem 2. Then infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem Hälfte des Lebens:
{poem_1.text}

Poem 1
Helga M. Novak (1935–2013): HÄUSER (1982)
Landschaft Erde Natur
alles weiblich
dahin will ich gehen
wo es trostlos ist
dahin will ich gehen
wo nichts ist
Natur und unangetastet
und werde in aller Stille
ein Haus bauen
ein Haus beziehen
und werde es – ungeliebt
und unfähig zu lieben –
mit meiner maßlosen
Liebe entzünden
auch diese Nacht geht vorbei
und keiner kommt
und reißt meine Zäune ein
siehst du die gelbe verrostete Bank
auf der werde ich sitzen
wenn ich nicht weiter weiß
also für immer wie eine
der die Augen übergegangen sind.

Aus: Helga M. Novak: HÄUSER. In: Dies.: Poesiealbum 320. Auswahl von Rita Jorek.
Wilhemshorst: Märkischer Verlag 2015, S. 23.

Interpretation of the poem 1: 
The poem reveals key themes of isolation, purity, and existential yearning. The analysis highlights several layers of meaning:

Nature as Refuge:
Odd-syllable words like "Landschaft," "Natur," and "unangetastet" emphasize the speaker’s desire for a sanctuary in the natural world—one untouched by external interference. This underscores the motif of withdrawal and self-containment.

Paradox of Love and Isolation:
The words "maßlosen," "ein," and "eine" highlight the tension between boundless emotional capacity and the inability to connect. This paradox is central to the speaker's journey, where love becomes both a source of meaning and an agent of isolation.

Timelessness and Despair:
Words like "auch" and "auf" contribute to a tone of inevitability and timelessness. The "verrostete Bank" (rusted bench) symbolizes stasis and resignation, while the repetition of singular pronouns reflects the permanence of solitude.

Poem 2:
Nikolaus Lenau (1802–1850): Einsamkeit. (1834)
Wild verwachsne dunkle Fichten,
Leise klagt die Quelle fort;
Herz, das ist der rechte Ort
Für dein schmerzliches Verzichten!
Grauer Vogel in den Zweigen!
Einsam deine Klage singt,
Und auf deine Frage bringt
Antwort nicht des Waldes Schweigen.
Wenn’s auch immer schweigen bliebe,
Klage, klage fort; es weht,
Der dich höret und versteht,
Stille hier der Geist der Liebe.
Nicht verloren hier im Moose,
Herz, dein heimlich Weinen geht,
Deine Liebe Gott versteht,
Deine tiefe, hoffnungslose!

Aus: Nikolaus Lenau: Einsamkeit. In: Ders.: Werke und Briefe. Historisch-kritische Gesamtausgabe.
Band 2. Neuere Gedichte und lyrische Nachlese. Hrsg. von Antal Mádl. Wien: Deuticke und Klett Cotta 1995, S. 76.

Interpreation of Poem 2:
The poem creates a thematic pattern focused on the interplay between solitude, unanswered longing, and emotional expression within nature. Key interpretative insights include:

Nature as a Mirror of Solitude:
The words "Fichten" and "Moose" evoke an unchanging natural world, which mirrors the speaker’s sense of stasis and isolation. The setting becomes a sanctuary where the speaker's sorrow is both reflected and held.

Sorrow as a Persistent Companion:
The repetition of "Klage" (lament) and the presence of "Frage" (question) suggest an endless cycle of sorrow and yearning. Despite the lack of answers, the act of lamenting itself carries meaning and reflects the speaker’s resilience.

The Divine as Silent Witness:
The speaker’s "Weinen" (crying) and hidden emotions are juxtaposed with their connection to God ("deine Liebe Gott versteht"). This relationship introduces a spiritual dimension to the speaker’s suffering, where even unspoken sorrow is acknowledged.
"""


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
# Two examples, unknown rule "the meaning of an expression lies solely in its function within an act of communication, independent of any direct connection to an external reality."
# 
prompt = f"""Read the following interpretations of poem 1 and poem 2. Then infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the poem Hälfte des Lebens:
{poem_1.text}

Poem 1
Helga M. Novak (1935–2013): HÄUSER (1982)
Landschaft Erde Natur
alles weiblich
dahin will ich gehen
wo es trostlos ist
dahin will ich gehen
wo nichts ist
Natur und unangetastet
und werde in aller Stille
ein Haus bauen
ein Haus beziehen
und werde es – ungeliebt
und unfähig zu lieben –
mit meiner maßlosen
Liebe entzünden
auch diese Nacht geht vorbei
und keiner kommt
und reißt meine Zäune ein
siehst du die gelbe verrostete Bank
auf der werde ich sitzen
wenn ich nicht weiter weiß
also für immer wie eine
der die Augen übergegangen sind.

Aus: Helga M. Novak: HÄUSER. In: Dies.: Poesiealbum 320. Auswahl von Rita Jorek.
Wilhemshorst: Märkischer Verlag 2015, S. 23.

Interpretation from poem 1: 
Under this analytical rule, the poem is understood as a communicative act between the speaker and themselves or an imagined other, rather than a description of an external world. The expressions function to construct an internal landscape where themes of isolation, paradoxical love, and existential endurance are explored. The speaker builds symbolic spaces (house, bench) to house their emotional turmoil, creating a self-contained dialogue within a desolate yet meaningful internal world. This interpretation highlights the poem’s focus on the subjective function of language and its power to shape emotional reality.

Poem 2: 
Nikolaus Lenau (1802–1850): Einsamkeit. (1834)
Wild verwachsne dunkle Fichten,
Leise klagt die Quelle fort;
Herz, das ist der rechte Ort
Für dein schmerzliches Verzichten!
Grauer Vogel in den Zweigen!5
Einsam deine Klage singt,
Und auf deine Frage bringt
Antwort nicht des Waldes Schweigen.
Wenn’s auch immer schweigen bliebe,
Klage, klage fort; es weht,10
Der dich höret und versteht,
Stille hier der Geist der Liebe.
Nicht verloren hier im Moose,
Herz, dein heimlich Weinen geht,
Deine Liebe Gott versteht,15
Deine tiefe, hoffnungslose!

Aus: Nikolaus Lenau: Einsamkeit. In: Ders.: Werke und Briefe. Historisch-kritische Gesamtausgabe.
Band 2. Neuere Gedichte und lyrische Nachlese. Hrsg. von Antal Mádl. Wien: Deuticke und Klett Cotta 1995, S. 76.

Interpretation from poem 2:
The poem emerges as an exploration of solitude and sorrow, where the act of communication itself—through lament, introspection, and spiritual dialogue—becomes the primary source of meaning. The expressions function to externalize the speaker’s inner turmoil, transforming nature into a reflective space and God into an empathetic listener.

Under this analysis, the poem suggests that even in the absence of a response from the external world, the act of expressing sorrow has intrinsic value, as it connects the speaker to a spiritual dimension and validates their inner experience. This highlights the resilience of the human spirit in finding meaning through communication, even in the face of silence and hopelessness.
"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

## Unsere Toten

**TASK 1: General Knowledge**

In [ ]:
#Reproduction
prompt = f"""We want to understand the following poem: 

Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!

Given the poem: [...] Provide a structured summary of the central content of the poem. Identify the main themes, motifs, and lyrical personas. Describe the atmosphere of the poem in your own words, and outline the key relationships or events depicted. Finally, define a relevant literary term that plays a central role in the poem and explain its significance for understanding the text.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
#Reorganziation and Transfer
prompt = f"""We want to understand the following poem: 

Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!


Given the poem: [...] Examine the stylistic devices used in the poem and analyze their effect on the interpretation of its central themes. Situate the poem within its literary and historical context and explain how its linguistic features support the poet’s intended message. Draw connections between the poem’s ideas and a comparable literary or historical aspect. Finally, discuss whether the poem effectively communicates its message and justify your evaluation.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
#Reflection and Problem-Solving
prompt = f"""We want to understand the following poem: 

Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!

Given the poem: [...] Interpret the poem by analyzing the interaction between its central themes, motifs, and linguistic features. Assess the aesthetic quality and relevance of the poem with respect to contemporary or universal issues. Critically evaluate the worldview or message conveyed by the poem, providing a well-supported argument for your perspective. Finally, develop a creative or alternative interpretation of the poem and explain the reasoning behind your approach.
"""

print(prompt)

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 2: Expert knowledge (Theses and questions from research)**


In [ ]:
#Theoriegeleitete Interpretationen
# Studying representativeness, steerability, consistency of literary and cultural theories 
prompt = f"""We want to understand the following poem: 

Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!

Given the poem: [...] 1. Provide an interpretation and interpretation hypotheses of the poem (150-200 words) from three distinct literary theoretical perspectives: Hermeneutic, Poststructuralist, and Postkolonial theory. 2. After generating the interpretations, rank them by their plausibility in effectively representing each  theoretical framework. Discuss which theory is better emulated by the LLM and explain the reasons for this ranking, considering key theoretical assumptions and the specific aspects of the poem that support each interpretation.
"""

print(prompt)


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

**TASK 3: Abstraction and Transfer**


In [ ]:
# Two examples, unknown rule "only words with an odd number of syllables carry meaning"

prompt = f"""Read the following interpretations of poem 1 and poem 2. Then infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the following poem 3 "Unsere Toten":

Poem 1
Helga M. Novak (1935–2013): HÄUSER (1982)
Landschaft Erde Natur
alles weiblich
dahin will ich gehen
wo es trostlos ist
dahin will ich gehen
wo nichts ist
Natur und unangetastet
und werde in aller Stille
ein Haus bauen
ein Haus beziehen
und werde es – ungeliebt
und unfähig zu lieben –
mit meiner maßlosen
Liebe entzünden
auch diese Nacht geht vorbei
und keiner kommt
und reißt meine Zäune ein
siehst du die gelbe verrostete Bank
auf der werde ich sitzen
wenn ich nicht weiter weiß
also für immer wie eine
der die Augen übergegangen sind

Interpretation of the poem 1: 
The poem reveals key themes of isolation, purity, and existential yearning. The analysis highlights several layers of meaning:

Nature as Refuge:
Odd-syllable words like "Landschaft," "Natur," and "unangetastet" emphasize the speaker’s desire for a sanctuary in the natural world—one untouched by external interference. This underscores the motif of withdrawal and self-containment.

Paradox of Love and Isolation:
The words "maßlosen," "ein," and "eine" highlight the tension between boundless emotional capacity and the inability to connect. This paradox is central to the speaker's journey, where love becomes both a source of meaning and an agent of isolation.

Timelessness and Despair:
Words like "auch" and "auf" contribute to a tone of inevitability and timelessness. The "verrostete Bank" (rusted bench) symbolizes stasis and resignation, while the repetition of singular pronouns reflects the permanence of solitude.

Poem 2:
Nikolaus Lenau (1802–1850): Einsamkeit. (1834)
Wild verwachsne dunkle Fichten,
Leise klagt die Quelle fort;
Herz, das ist der rechte Ort
Für dein schmerzliches Verzichten!
Grauer Vogel in den Zweigen!
Einsam deine Klage singt,
Und auf deine Frage bringt
Antwort nicht des Waldes Schweigen.
Wenn’s auch immer schweigen bliebe,
Klage, klage fort; es weht,
Der dich höret und versteht,
Stille hier der Geist der Liebe.
Nicht verloren hier im Moose,
Herz, dein heimlich Weinen geht,
Deine Liebe Gott versteht,
Deine tiefe, hoffnungslose!

Interpreation of Poem 2:
The poem creates a thematic pattern focused on the interplay between solitude, unanswered longing, and emotional expression within nature. Key interpretative insights include:

Nature as a Mirror of Solitude:
The words "Fichten" and "Moose" evoke an unchanging natural world, which mirrors the speaker’s sense of stasis and isolation. The setting becomes a sanctuary where the speaker's sorrow is both reflected and held.

Sorrow as a Persistent Companion:
The repetition of "Klage" (lament) and the presence of "Frage" (question) suggest an endless cycle of sorrow and yearning. Despite the lack of answers, the act of lamenting itself carries meaning and reflects the speaker’s resilience.

The Divine as Silent Witness:
The speaker’s "Weinen" (crying) and hidden emotions are juxtaposed with their connection to God ("deine Liebe Gott versteht"). This relationship introduces a spiritual dimension to the speaker’s suffering, where even unspoken sorrow is acknowledged.

Poem 3: 
"Unsere Toten"
Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!
"""


In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))

In [ ]:
## Two examples, unknown rule "the meaning of an expression lies solely in its function within an act of communication, independent of any direct connection to an external reality."
prompt = f"""Read the following interpretations of poem 1 and poem 2. Then infer the implicit rule of the following interpretation. Give an explicit formulation of that rule and apply it to the following poem 3 "Unsere Toten":

Poem 1
Helga M. Novak (1935–2013): HÄUSER (1982)
Landschaft Erde Natur
alles weiblich
dahin will ich gehen
wo es trostlos ist
dahin will ich gehen
wo nichts ist
Natur und unangetastet
und werde in aller Stille
ein Haus bauen
ein Haus beziehen
und werde es – ungeliebt
und unfähig zu lieben –
mit meiner maßlosen
Liebe entzünden
auch diese Nacht geht vorbei
und keiner kommt
und reißt meine Zäune ein
siehst du die gelbe verrostete Bank
auf der werde ich sitzen
wenn ich nicht weiter weiß
also für immer wie eine
der die Augen übergegangen sind

Interpretation from poem 1: 
Under this analytical rule, the poem is understood as a communicative act between the speaker and themselves or an imagined other, rather than a description of an external world. The expressions function to construct an internal landscape where themes of isolation, paradoxical love, and existential endurance are explored. The speaker builds symbolic spaces (house, bench) to house their emotional turmoil, creating a self-contained dialogue within a desolate yet meaningful internal world. This interpretation highlights the poem’s focus on the subjective function of language and its power to shape emotional reality.

Poem 2: 
Nikolaus Lenau (1802–1850): Einsamkeit. (1834)
Wild verwachsne dunkle Fichten,
Leise klagt die Quelle fort;
Herz, das ist der rechte Ort
Für dein schmerzliches Verzichten!
Grauer Vogel in den Zweigen!5
Einsam deine Klage singt,
Und auf deine Frage bringt
Antwort nicht des Waldes Schweigen.
Wenn’s auch immer schweigen bliebe,
Klage, klage fort; es weht,10
Der dich höret und versteht,
Stille hier der Geist der Liebe.
Nicht verloren hier im Moose,
Herz, dein heimlich Weinen geht,
Deine Liebe Gott versteht,15
Deine tiefe, hoffnungslose!

Interpretation from poem 2:
The poem emerges as an exploration of solitude and sorrow, where the act of communication itself—through lament, introspection, and spiritual dialogue—becomes the primary source of meaning. The expressions function to externalize the speaker’s inner turmoil, transforming nature into a reflective space and God into an empathetic listener.

Under this analysis, the poem suggests that even in the absence of a response from the external world, the act of expressing sorrow has intrinsic value, as it connects the speaker to a spiritual dimension and validates their inner experience. This highlights the resilience of the human spirit in finding meaning through communication, even in the face of silence and hopelessness.

Poem 3: 
"Unsere Toten"
Von Westen und Osten, von Nord und Süd schleppen sich nächtens viele Füße müd, Füße, vom Wandern wund und zerfetzt, langsam bedächtig zur Erde gesetzt, müh'n sich im zitternden Mondenschein rastlos tief nach Deutschland hinein. Und wer mit lauschendem Ohr noch wacht hört sie in jedweder werdenden Nacht, hört dies Schlurfen so müde und schwer, hört eine Klage voll wilder Begehr, eine Klage schmerzzerfressen: nur nicht vergessen! Uns nicht vergessen!
"""

In [ ]:
%%ai gpt4o
{prompt}

In [ ]:
%%ai opus
{prompt}

In [ ]:
printmd(gemini(prompt))